In [24]:
import numpy as np
import pandas as pd



In [25]:
from pathlib import Path
import sys, os
from typing import Dict, List, NamedTuple, Tuple
import numpy as np
from collections import defaultdict
import time


In [26]:
def compute_metrics(alg_name, groups, results):
    # test_data are triplets: user_id, item_id, and rating
    #LP: test data is matrix user_id x item_id !!!!!! a ja si rikal, jakto ze ti to prirazeni funguje...

    avg_rating = []
    min_rating = []
    minmax_rating = []
    std_rating = []
    
    avg_nDCG_rating = []
    min_nDCG_rating = []
    minmax_nDCG_rating = []
    std_nDCG_rating = []
        
    for gid in groups:
        gid_res_ndcg = results.loc[(results.group_id == gid)&(results.metric == "nDCG")]
        gid_res_AR = results.loc[(results.group_id == gid)&(results.metric == "AR")] 
        
        #print(gid_res_ndcg)
        #print(gid_res_AR)
        
        group_users_mean_ratings = [i for i in gid_res_AR.result]
        group_users_ndcg_ratings = [i for i in gid_res_ndcg.result]
        
        #print(group_users_mean_ratings)
        if len(group_users_mean_ratings) > 0:
            avg_rating.append(np.average(group_users_mean_ratings)) 
            min = np.min(group_users_mean_ratings)
            min_rating.append(min) 
            max = np.max(group_users_mean_ratings)
            minmax_rating.append(0.0 if max == 0.0 else min/max)
            std_rating.append(np.std(group_users_mean_ratings)) 

            avg_nDCG_rating.append(np.average(group_users_ndcg_ratings)) 
            min = np.min(group_users_ndcg_ratings)
            min_nDCG_rating.append(min) 
            max = np.max(group_users_ndcg_ratings)
            minmax_nDCG_rating.append(0.0 if max == 0.0 else min/max)
            std_nDCG_rating.append(np.std(group_users_ndcg_ratings)) 

    print(len(avg_rating))    
    results = pd.DataFrame({
       "AR_avg": avg_rating ,
        "AR_min": min_rating ,
        "AR_min/max": minmax_rating ,
        #"AR_std": std_rating ,
        
        "nDCG_avg": avg_nDCG_rating ,
        "nDCG_min": min_nDCG_rating ,
        "nDCG_min/max": minmax_nDCG_rating #,
        #"nDCG_std": std_nDCG_rating 
    })
      
    return results

In [28]:
from scipy import stats
#Comparison for size 8
def calculate_significance(results, grouptype, size, algs):
    print("")
    print(grouptype,size)

    otherAlgs = [a for a in algs if ( grouptype+"_group_"+size in a )]

    for count1, alg1 in enumerate(otherAlgs):
        baseDF = results[alg1]
        
        for count2,alg in enumerate(otherAlgs):
            if count1 != count2:
                otherDF = results[alg]
                res = {}
                for col in otherDF.columns:
                    mean_first = np.mean(baseDF.loc[:,col])
                    mean_second = np.mean(otherDF.loc[:,col])
                    mean_diff = np.mean(baseDF.loc[:,col] - otherDF.loc[:,col])
                    stat,pval = stats.ttest_rel(baseDF.loc[:,col], otherDF.loc[:,col])
                    if pval > 0.0001:
                        print("pval_excess",col)
                    res[col] = (mean_first,mean_second,mean_diff, pval)

                alg1 = alg1.replace("mf_230_1.0_avg_","")
                alg1 = alg1.replace(grouptype+"_group_"+size+"_","")
                alg = alg.replace("mf_230_1.0_avg_","")
                alg = alg.replace(grouptype+"_group_"+size+"_","")
                print(alg1+" - "+ alg)
                print(pd.DataFrame(res, index=["first","second","diff","pval"]))
                print("")

In [29]:
results_all = {}
algs_sign = []
whitelist = ['GFAR', '_avg20', 'FuzzyDHondtDirectOptimize_1', 'GreedyLM',  'FuzzyDHondt_1',  'SPGreedy',  'fai',  'xpo']
blacklist= "weighted"
for grouptype in ["sim","div"]:
    for size in ["2","3","4","8"]:
        fileName = "../../evaluation_kgrec/results/resultRaw_"+grouptype+"_"+size
        resultsDF = pd.read_csv(fileName, sep=",", header=None, names=["alg","group_id","user_id","metric","result"])
        algs = resultsDF.alg.unique()        
        metrics = resultsDF.metric.unique()
        groups = resultsDF.group_id.unique()        
        results = {}
        for a in algs:
            for item in whitelist:
                if item in a and "mf_data" not in a:
                    algs_sign.append(a)
                    resPerAlg = resultsDF.loc[resultsDF.alg == a]            
                    results[a] = compute_metrics(a, groups, resPerAlg)
        calculate_significance(results, grouptype, size, algs_sign)

1775
1775
1775
1775
1775
1775
1775
1775

sim 2
mf_230_1.0_avg20_sim_group_2 - FuzzyDHondtDirectOptimize_1.0
          AR_avg         AR_min  AR_min/max  nDCG_avg      nDCG_min  \
first   0.458781   3.935638e-01    0.764413  0.740805  6.808066e-01   
second  0.439821   4.362622e-01    0.983999  0.721716  6.889433e-01   
diff    0.018960  -4.269838e-02   -0.219585  0.019088 -8.136656e-03   
pval    0.000000  1.393030e-263    0.000000  0.000000  9.719008e-17   

         nDCG_min/max  
first    8.534938e-01  
second   9.154741e-01  
diff    -6.198032e-02  
pval    3.319653e-141  

mf_230_1.0_avg20_sim_group_2 - FuzzyDHondt_1.0
               AR_avg         AR_min  AR_min/max      nDCG_avg       nDCG_min  \
first    4.587813e-01   3.935638e-01    0.764413  7.408047e-01   6.808066e-01   
second   4.571301e-01   4.257578e-01    0.876682  7.417764e-01   7.219085e-01   
diff     1.651199e-03  -3.219403e-02   -0.112269 -9.716542e-04  -4.110186e-02   
pval    1.415745e-147  1.723788e-307    0.00

GFAR_1.0 - GreedyLM_1.0
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   0.413003  0.387873    0.888098  0.674157  0.648623  9.273366e-01
second  0.440176  0.414177    0.890936  0.719356  0.695043  9.355582e-01
diff   -0.027173 -0.026304   -0.002838 -0.045199 -0.046420 -8.221616e-03
pval    0.000000  0.000000    0.018524  0.000000  0.000000  1.089556e-10

GFAR_1.0 - SPGreedy_1.0
          AR_avg        AR_min     AR_min/max  nDCG_avg      nDCG_min  \
first   0.413003  3.878732e-01   8.880981e-01  0.674157  6.486229e-01   
second  0.458482  3.958412e-01   7.717514e-01  0.718487  6.600260e-01   
diff   -0.045479 -7.968054e-03   1.163466e-01 -0.044330 -1.140310e-02   
pval    0.000000  4.737434e-14  3.309204e-207  0.000000  2.049320e-15   

         nDCG_min/max  
first    9.273366e-01  
second   8.525865e-01  
diff     7.475008e-02  
pval    1.665864e-140  

GFAR_1.0 - mf_230_1.0_fai_sim_group_2
              AR_avg        AR_min    AR_min/max  nDCG_avg   

mf_230_1.0_xpo_sim_group_2 - FuzzyDHondtDirectOptimize_1.0
               AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min   nDCG_min/max
first    4.143336e-01  0.388567    0.885948  0.622259  0.571602   8.530262e-01
second   4.398212e-01  0.436262    0.983999  0.721716  0.688943   9.154741e-01
diff    -2.548755e-02 -0.047696   -0.098051 -0.099457 -0.117341  -6.244794e-02
pval    4.564093e-291  0.000000    0.000000  0.000000  0.000000  3.376189e-106

mf_230_1.0_xpo_sim_group_2 - FuzzyDHondt_1.0
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min   nDCG_min/max
first   0.414334  0.388567    0.885948  0.622259  0.571602   8.530262e-01
second  0.457130  0.425758    0.876682  0.741776  0.721908   9.484971e-01
diff   -0.042796 -0.037191    0.009266 -0.119517 -0.150306  -9.547098e-02
pval    0.000000  0.000000    0.000037  0.000000  0.000000  7.871983e-250

pval_excess AR_min
pval_excess AR_min/max
mf_230_1.0_xpo_sim_group_2 - GFAR_1.0
              AR_avg    AR_min  AR_min/max  nDCG_

pval_excess AR_min/max
GFAR_1.0 - FuzzyDHondt_1.0
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   0.336758  0.301449    0.814073  0.550988  0.483793  7.948884e-01
second  0.397302  0.354978    0.812537  0.634228  0.578006  8.412207e-01
diff   -0.060544 -0.053530    0.001536 -0.083240 -0.094213 -4.633222e-02
pval    0.000000  0.000000    0.555106  0.000000  0.000000  8.966858e-69

GFAR_1.0 - GreedyLM_1.0
          AR_avg    AR_min    AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   0.336758  0.301449  8.140727e-01  0.550988  0.483793  7.948884e-01
second  0.384899  0.340268  7.962282e-01  0.614660  0.560494  8.408916e-01
diff   -0.048141 -0.038819  1.784444e-02 -0.063672 -0.076701 -4.600315e-02
pval    0.000000  0.000000  1.450456e-14  0.000000  0.000000  6.985927e-73

GFAR_1.0 - SPGreedy_1.0
          AR_avg         AR_min     AR_min/max  nDCG_avg       nDCG_min  \
first   0.336758   3.014487e-01   8.140727e-01  0.550988   4.837934e-01   
second  0.

pval    0.000000  0.000000    0.006539  0.000000  0.000000  2.178349e-184

mf_230_1.0_fai_sim_group_3 - SPGreedy_1.0
          AR_avg         AR_min    AR_min/max  nDCG_avg  nDCG_min  \
first   0.329157   2.894480e-01  7.887653e-01  0.507261  0.431306   
second  0.395931   3.270593e-01  7.088005e-01  0.603760  0.520559   
diff   -0.066774  -3.761128e-02  7.996478e-02 -0.096499 -0.089252   
pval    0.000000  1.105090e-203  7.302707e-94  0.000000  0.000000   

        nDCG_min/max  
first   7.436216e-01  
second  7.637944e-01  
diff   -2.017276e-02  
pval    3.387229e-08  

pval_excess AR_min/max
pval_excess nDCG_min
pval_excess nDCG_min/max
mf_230_1.0_fai_sim_group_3 - mf_230_1.0_xpo_sim_group_3
               AR_avg        AR_min  AR_min/max      nDCG_avg  nDCG_min  \
first    3.291569e-01  2.894480e-01    0.788765  5.072609e-01  0.431306   
second   3.420745e-01  3.021550e-01    0.796164  5.111802e-01  0.435581   
diff    -1.291761e-02 -1.270698e-02   -0.007398 -3.919259e-03 -0.004275

...

GFAR_1.0 - SPGreedy_1.0
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   0.329201  0.299901    0.840917  0.583219  0.549216      0.890984
second  0.356727  0.260866    0.608132  0.611933  0.497196      0.700250
diff   -0.027526  0.039035    0.232786 -0.028714  0.052020      0.190735
pval    0.000000  0.000000    0.000000  0.000000  0.000000      0.000000

pval_excess AR_min/max
pval_excess nDCG_min/max
GFAR_1.0 - mf_230_1.0_fai_div_group_2
              AR_avg        AR_min  AR_min/max  nDCG_avg      nDCG_min  \
first   3.292006e-01  2.999015e-01    0.840917  0.583219  5.492159e-01   
second  3.288796e-01  2.995917e-01    0.840815  0.577067  5.422881e-01   
diff    3.209652e-04  3.097915e-04    0.000102  0.006152  6.927868e-03   
pval    1.802354e-30  4.256675e-07    0.738665  0.000000  4.074361e-44   

        nDCG_min/max  
first       0.890984  
second      0.888677  
diff        0.002308  
pval        0.078224  

pval_excess AR_avg
pval_excess AR_min

mf_230_1.0_xpo_div_group_2 - mf_230_1.0_avg20_div_group_2
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   0.329233  0.299990    0.841202  0.546158  0.493098      0.828092
second  0.356925  0.259076    0.602931  0.618873  0.486202      0.667390
diff   -0.027692  0.040914    0.238272 -0.072715  0.006896      0.160702
pval    0.000000  0.000000    0.000000  0.000000  0.000040      0.000000

mf_230_1.0_xpo_div_group_2 - FuzzyDHondtDirectOptimize_1.0
              AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   3.292332e-01  0.299990    0.841202  0.546158  0.493098  8.280920e-01
second  3.304769e-01  0.326312    0.975462  0.592029  0.549076  8.701328e-01
diff   -1.243725e-03 -0.026322   -0.134259 -0.045871 -0.055978 -4.204081e-02
pval    1.369396e-08  0.000000    0.000000  0.000000  0.000000  7.436892e-96

mf_230_1.0_xpo_div_group_2 - FuzzyDHondt_1.0
          AR_avg    AR_min    AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   0.3

pval    0.000000  0.000000  1.350266e-48  0.000000  0.000000  1.664440e-35

GFAR_1.0 - SPGreedy_1.0
          AR_avg         AR_min  AR_min/max  nDCG_avg       nDCG_min  \
first   0.246506   2.079507e-01    0.738108  0.437409   3.797333e-01   
second  0.281572   1.838850e-01    0.501693  0.476063   3.509102e-01   
diff   -0.035067   2.406572e-02    0.236414 -0.038655   2.882309e-02   
pval    0.000000  4.453795e-280    0.000000  0.000000  1.173773e-156   

        nDCG_min/max  
first       0.774304  
second      0.598997  
diff        0.175307  
pval        0.000000  

pval_excess AR_min/max
GFAR_1.0 - mf_230_1.0_fai_div_group_3
          AR_avg        AR_min  AR_min/max  nDCG_avg       nDCG_min  \
first   0.246506  2.079507e-01    0.738108  0.437409   3.797333e-01   
second  0.244115  2.059517e-01    0.736519  0.423077   3.584122e-01   
diff    0.002391  1.999012e-03    0.001588  0.014331   2.132109e-02   
pval    0.000000  5.095251e-17    0.225276  0.000000  1.563981e-260   

      

pval_excess AR_min/max
mf_230_1.0_fai_div_group_3 - mf_230_1.0_xpo_div_group_3
              AR_avg        AR_min  AR_min/max  nDCG_avg       nDCG_min  \
first   2.441147e-01  2.059517e-01    0.736519  0.423077   3.584122e-01   
second  2.455192e-01  2.072774e-01    0.738141  0.411393   3.387522e-01   
diff   -1.404505e-03 -1.325712e-03   -0.001622  0.011684   1.965999e-02   
pval    7.288713e-71  1.386008e-07    0.246780  0.000000  2.413172e-172   

        nDCG_min/max  
first   7.371817e-01  
second  6.992446e-01  
diff    3.793719e-02  
pval    1.915453e-71  

mf_230_1.0_xpo_div_group_3 - mf_230_1.0_avg20_div_group_3
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   0.245519  0.207277    0.738141  0.411393  0.338752      0.699245
second  0.282684  0.177805    0.478726  0.490177  0.332019      0.533148
diff   -0.037165  0.029472    0.259415 -0.078783  0.006733      0.166096
pval    0.000000  0.000000    0.000000  0.000000  0.000003      0.000000

mf_23

KeyboardInterrupt: 